In [5]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd

In [2]:
all_letters = string.printable
n_letters = len(all_letters)
def letterToIndex(letter):
    return all_letters.find(letter)
def linesToTensor(lines):
    line_length = 15000
    if max([ len(line) for line in lines]) < line_length:
        line_length = max( [len(line) for line in lines] )
    #line_length = max( [len(line) for line in lines] )
    #xx = [max(len(line)-15000,0) for line in lines]
    #print float(np.sum(xx)) / float(np.sum([len(line) for line in lines]))
    tensor = torch.zeros(len(lines), line_length, n_letters)
    for b, line in enumerate(lines): 
        line = line[:15000]
        for li, letter in enumerate(line):
            tensor[b][li + line_length - len(line)][letterToIndex(letter)] = 1

    return tensor

In [3]:
class LangModel(nn.Module):
    def __init__(self, preTrained='True', input=100):
        super(LangModel, self).__init__()

        # Language Model
        self.lang = nn.LSTM(input, 128, 3, batch_first=True) 
 
        # Output 
        self.output = nn.Linear(128, 2)
        n = self.output.in_features * self.output.out_features
        self.output.weight.data.normal_(0, math.sqrt(2. / n))
        self.output.bias.data.zero_()

    def forward(self, text):
        text.cuda()
        h0 = ( Variable(torch.zeros(3, text.size(0), 128)).cuda(),  Variable(torch.zeros(3, text.size(0), 128)).cuda())

        lang_feature, hn = self.lang(text, h0 )
        lang_feature = lang_feature[:,-1,:]

        pred = self.output(lang_feature)
        return pred


In [4]:
"""
chat_data='./exp_chat/{}.npy'.format(game_id)
chat_arr=np.load(chat_data)
window_text=[]
for i in range(len(chat_arr)):
    temp=''
    for idx in range(7):
        if i+idx<len(chat_arr):
            temp+=chat_arr[i+idx]
    window_text.append(temp)
"""

NameError: name 'game_id' is not defined

In [54]:
text=linesToTensor(window_text)

In [58]:
text.size()

torch.Size([2056, 2700, 100])

In [11]:
import torch.utils.data as data

class chat_ds(data.Dataset):
    def __init__(self,game_id):
        self.window_text=[]
        #####game id 전체로 바꿀것##############
        chat_data='./exp_chat/{}.npy'.format(game_id)
        chat_arr=np.load(chat_data)
        #window_text=np.zeros([len(text)])
        for i in range(len(chat_arr)):
            temp=''
            for idx in range(7):
                if i+idx<len(chat_arr):
                    temp+=chat_arr[i+idx]
            self.window_text.append(temp)
        self.text=linesToTensor(self.window_text)

        with open('../tsvt/ocr/hi_time.json','r') as f:
            labels=json.load(f)

        label=labels[game_id]
        
        self.label_gt=torch.zeros((len(self.text),))
        for (x,y) in label:
            start=int(x.split(':')[0])*60+int(x.split(':')[1])
            end=int(y.split(':')[0])*60+int(y.split(':')[1])
            for i in range(start,end+1):
                self.label_gt[i]=1
        self.label_gt=self.label_gt.long()
    def __len__(self):
        return len(self.label_gt)
    def __getitem__(self,index):
        return self.text[index],self.label_gt[index]



In [12]:
game_id='102844212428895431'
a=chat_ds(game_id)

In [13]:
loader=torch.utils.data.DataLoader(a,batch_size=100)

In [14]:
###### model load #####
model=LangModel().cuda()
criterion = nn.CrossEntropyLoss().cuda()

In [93]:
#train set
for iter in range(10):
    model.train()
    for inputs,labels in (loader):
        inputs = inputs.cuda()
        labels = labels.cuda()
        optimizer = torch.optim.SGD(model.parameters(), 0.1)
        optimizer.zero_grad()
        output=model(inputs)

        loss=criterion(output,labels)
        loss.backward()
        optimizer.step()

    #validation
    model.eval()
    pred_sum = 0#model output
    gt_sum = 0#label
    tp_sum=0
    fp_sum=0
    fn_sum=0
    with torch.no_grad():
        for it, (inputs,labels) in enumerate(loader):
            count+=1
            inputs = inputs.cuda()
            labels = labels.cuda()
            output=model(inputs)

            TP,FP,TN,FN,pred_len, gt_len=fmeasure(output.cpu(),labels.cpu())
            tp_sum += TP
            fp_sum += FP
            fn_sum += FN
            pred_sum += pred_len
            gt_sum += gt_len

        precision = tp_sum/(tp_sum+fp_sum)
        recall = tp_sum / (tp_sum+fn_sum)
        f1 = (2*precision*recall / (precision + recall)) * 100
        print( tp_sum, fp_sum, fn_sum)
        print('[{}/{}], prec:{}, recall:{}, f1:{}'.format(it, len(loader), precision, recall, f1))

923 1133 0
[20/21], prec:0.4489299610894942, recall:1.0, f1:61.967103054716354
923 1133 0
[20/21], prec:0.4489299610894942, recall:1.0, f1:61.967103054716354


KeyboardInterrupt: 

In [88]:
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len

## Chat data .npy loader

In [35]:
data=[]
for idx,gi in enumerate(os.listdir("./result")):
        print(gi)
        chat_data=pd.read_csv('./result/'+gi)
        delay=int(gi.split('_')[1])
        duration=float(gi.split('_')[2][:-4])
        gameid=int(gi.split('_')[0])
        meta=metadata.loc[metadata['gameid']==gameid]
        
        st=meta['new_st']
        et=meta['new_et']
        
        data=['' for i in range(math.floor(duration)+math.floor(delay)+1)]


        init_time=int(st)-int(delay)
        for c_idx,c in enumerate(chat_data['duration']):
            data[int(c)-init_time]=data[int(c)-init_time]+str(chat_data['chat'][c_idx])+'\n'
        data=np.array(data)
        np.save('./exp_chat/{}'.format(gi),data)


102844412704890154_0_2079.0.csv
102844224147717245_0_1836.000000000001.csv
102844235748703677_0_1829.0.csv
102844412723567946_0_2159.0.csv
102844412708953395_0_2060.0.csv
102844212429092040_179_2254.0.csv
102844341908026005_0_2917.999999999998.csv
102844235753749959_0_2218.0.csv
102844294674286796_0_2680.0.csv
102844412721339716_0_1821.0.csv
102844341906256529_0_1751.0000000000018.csv
102844412722519367_0_2072.0.csv
102844235747982779_0_2731.0.csv
102844412716686654_0_1647.0.csv
102844401154168486_0_1924.0.csv
102844412704496937_0_2632.0.csv
102844212430075086_0_2282.999999999999.csv
102844212429288649_0_2922.0.csv
102844235750997440_1_1675.0.csv
102844294670026952_0_1708.0.csv
102844224148896895_0_1807.0.csv
102844294669109446_0_1562.0.csv
102844212431516886_0_1821.0.csv
102844294674876621_0_1718.0.csv
102844401154430631_0_1655.0000000000036.csv
102844401151874719_0_2103.0.csv
102844412712164667_0_2088.0.csv
102844294670878922_0_2199.0.csv
102844412705545516_0_1541.0.csv
1028444127052

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [29]:
os.listdir("./result")
#metadata=pd.read_csv('new_metadata.csv')

['102844412704890154_0_2079.0.csv',
 '102844224147717245_0_1836.000000000001.csv',
 '102844235748703677_0_1829.0.csv',
 '102844412723567946_0_2159.0.csv',
 '102844412708953395_0_2060.0.csv',
 '102844212429092040_179_2254.0.csv',
 '102844341908026005_0_2917.999999999998.csv',
 '102844235753749959_0_2218.0.csv',
 '102844294674286796_0_2680.0.csv',
 '102844412721339716_0_1821.0.csv',
 '102844341906256529_0_1751.0000000000018.csv',
 '102844412722519367_0_2072.0.csv',
 '102844235747982779_0_2731.0.csv',
 '102844412716686654_0_1647.0.csv',
 '102844401154168486_0_1924.0.csv',
 '102844412704496937_0_2632.0.csv',
 '102844212430075086_0_2282.999999999999.csv',
 '102844212429288649_0_2922.0.csv',
 '102844235750997440_1_1675.0.csv',
 '102844294670026952_0_1708.0.csv',
 '102844224148896895_0_1807.0.csv',
 '102844294669109446_0_1562.0.csv',
 '102844212431516886_0_1821.0.csv',
 '102844294674876621_0_1718.0.csv',
 '102844401154430631_0_1655.0000000000036.csv',
 '102844401151874719_0_2103.0.csv',
 '102

In [12]:
metadata.loc[metadata['gameid']==102844212430271695]['new_st']

1    9197
Name: new_st, dtype: int64

In [10]:
metadata

,Unnamed: 0,gameid,urlid,st,et,intime,video time,new_st,new_et
0,0,102844212430075086,489172130,2019-10-02T11:13:18.688Z,2019-10-02T11:51:21.804Z,3:38,1:29:51,5173,7456.0
1,1,102844212430271695,489172130,2019-10-02T12:20:25.129Z,2019-10-02T12:54:13.187Z,5:13,2:38:30,9197,11225.0
2,2,102844212429092040,489172130,2019-10-02T13:20:18.470Z,2019-10-02T13:57:53.281Z,4:57,3:38:07,12790,15044.0
3,3,102844212430402768,489172130,2019-10-02T14:23:58.095Z,2019-10-02T15:03:26.737Z,6:19,4:43:09,16610,18978.0
4,4,102844212430599377,489172130,2019-10-02T15:28:36.590Z,2019-10-02T16:07:36.613Z,1:52,5:43:23,20491,22831.0
...,...,...,...,...,...,...,...,...,...
107,107,102844412723174729,503346504,2019-11-03T13:08:27.814Z,2019-11-03T13:42:07.720Z,2:20,3:47:27,13507,15526.0
108,108,102844412722519367,503346504,2019-11-03T11:15:39.545Z,2019-11-03T11:50:11.667Z,0:29,1:52:48,6739,8811.0
109,109,102844412720618818,506331985,2019-11-10T12:40:36.099Z,2019-11-10T13:30:28.249Z,0:36,8:45:29,31493,34485.0
110,110,102844412721339716,506331985,2019-11-10T14:49:26.226Z,2019-11-10T15:19:48.032Z,0:36,10:45:21,38685,40506.0


In [37]:
if __name__ == "__main__":
    chat_data='./exp_chat/102844235747982779_0_2731.0.csv.npy'
    chat_arr=np.load(chat_data)

In [38]:
chat_arr

array(['WutFace\n444444444444\nHOOOOOOOOOOOOOOOOK\nResidentSleeper a\nFree Hong Kong\n',
       'WutFace\nEU PogChamp\nUOL Wins\nEU FANS WutFace\nPVMAN = GG PepeHands PVMAN = GG PepeHands PVMAN = GG\nANELE ANELE ANELE\nUOL 3-1 CLAP\nmonkaW\nEU WILL WIN\n',
       '🐍 RapThis 🦄\nLOCH NESS MONSTER WutFace\nEU crowd monkaS\nFree Hong Kong\nWutFace\nSwiftRage FREE NBA SwiftRage FREE BLIZZARD SwiftRage FREE HONG KONG\n',
       ...,
       'WutFace\nFree HK ---------------- Free NBA ------------- Free Blizzard\nGG EZ\nLUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU\nGG\nOH MY GOSH THIS FANS ARE ANNOYING AF\nGG\ngg Pog\nEU LUL\nrooREE rooREE rooREE rooREE\nGG\nEU LUL\nEZ Clap\nKEKW\nGG\n',
       'WutFace !!\nSwiftRage UNICORNS\n', ''], dtype='<U835')